In [2]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 6.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import requests
import io
import os
from groq import Groq
from IPython.display import display, HTML

In [28]:
from google.colab import userdata

# Fetch Groq API key from Colab's Secrets Manager.
# Make sure you've added the secret named 'GROQ_API_KEY' in the left-hand sidebar.
try:
    groq_api_key = userdata.get("GROQ_API_KEY")
except userdata.SecretNotFoundError:
    raise ValueError("GROQ_API_KEY not found in Colab secrets. Please set it.")

# Initialize the Groq client with the fetched API key.
# Passing the key directly ensures it's used correctly.
client = Groq(api_key=groq_api_key)

In [32]:
def clean_data_with_llm(file_path):
    """
    Reads a CSV, prompts an LLM to clean it, and returns the cleaned data
    and a summary of changes.
    """
    try:
        # Step A: Pre-analysis to find the correct delimiter
        delimiters = [',', ';', '\t', '|']
        df_original = None
        delimiter_found = None
        for delimiter in delimiters:
            try:
                # Use `on_bad_lines='skip'` to handle potential parsing errors in some rows
                df_original = pd.read_csv(file_path, delimiter=delimiter, on_bad_lines='skip')
                delimiter_found = delimiter
                print(f"File '{file_path}' successfully read with delimiter: '{delimiter_found}'")
                break
            except pd.errors.ParserError as e:
                print(f"Failed to read with delimiter '{delimiter}': {e}")
                continue

        if df_original is None:
            raise ValueError("Could not read the CSV file with common delimiters.")

        # Step B: Prepare the prompt for the LLM
        sample_data = df_original.head(50).to_csv(index=False)

        prompt = f"""
        You are a data cleaning agent. Your task is to clean a dataset based on common errors.
        The following dataset contains errors such as:
        - missing values
        - inconsistent date formats
        - duplicates
        - inconsistent casing in text (e.g., 'Rishita' vs 'rishita').
        - Incorrect data types for columns.

        Here is a sample of the dataset (first 50 rows) from a file named '{os.path.basename(file_path)}':
        {sample_data}

        1. First, **analyze and clean this dataset**. Return the corrected version in valid CSV format (same columns as input).
        The CSV output must be directly after this prompt and not enclosed in any code block. Do not include any extra text before the CSV data.

        2. **After the CSV data**, provide a brief summary of the cleaning steps you performed. Start the summary with the markdown heading `### Cleaning Summary`.

        Begin by returning the cleaned CSV directly.
        """

        # Step C: Call the Groq API
        print(f"\nSending '{os.path.basename(file_path)}' to Groq for cleaning...")
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama-3.1-8b-instant",
            temperature=0.0
        )

        full_response = chat_completion.choices[0].message.content

        # Step D: Extract the cleaned CSV and the cleaning summary
        parts = full_response.split('### Cleaning Summary')
        if len(parts) < 2:
            print("Warning: The model did not provide a cleaning summary as expected.")
            csv_data = full_response
            cleaning_summary = "No summary provided by the model."
        else:
            csv_data = parts[0].strip()
            cleaning_summary = "### Cleaning Summary" + parts[1].strip()

        # Step E: Load the cleaned data into a new DataFrame
        cleaned_df = pd.read_csv(io.StringIO(csv_data))

        return df_original, cleaned_df, cleaning_summary

    except Exception as e:
        print(f"An error occurred while cleaning '{file_path}': {e}")
        return None, None, None

In [36]:
file_paths = ["GDP.csv", "Grocery_Inventory.csv", "OnlineRetail.csv"]

In [37]:
for file_path in file_paths:
    print(f"\n{'='*20} Cleaning {file_path} {'='*20}\n")
    original_df, cleaned_df, log_summary = clean_data_with_llm(file_path)

    if original_df is not None and cleaned_df is not None:
        print("--- Cleaning Log ---")
        print(f"File: {file_path}")
        print(log_summary)

        print("\n--- Original Data (First 10 Rows) ---")
        print(original_df.head(10).to_string())

        print("\n--- Cleaned Data (First 10 Rows) ---")
        print(cleaned_df.head(10).to_string())



==================== Cleaning GDP.csv ====================

File 'GDP.csv' successfully read with delimiter: ','

Sending 'GDP.csv' to Groq for cleaning...
--- Cleaning Log ---
File: GDP.csv
### Cleaning SummaryThe dataset was cleaned by performing the following steps:

- **Handling missing values**: Missing values were replaced with 0.0 for numeric columns and an empty string for text columns.
- **Date format standardization**: The date format was standardized to YYYY.0 for all columns.
- **Duplicate removal**: Duplicates were removed from the dataset.
- **Text casing standardization**: Text casing was standardized to lowercase for all text columns.
- **Data type correction**: Data types were corrected for all columns to match the expected data type.
- **Data validation**: Data was validated to ensure that it falls within the expected range for each column.

--- Original Data (First 10 Rows) ---
          Country  IMF_Forecast  IMF_Year  WorldBank_Estimate  WorldBank_Year  UN_Estimat